In [ ]:
%matplotlib notebook
from random import randint

import os
import time
import tensorflow as tf
import setproctitle as SP
import pandas as pd
import numpy as np
import datetime
import csv
from pandas import DataFrame
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import scipy.io
from sklearn.model_selection import train_test_split
from numpy import savetxt
from numpy import genfromtxt
from sklearn.preprocessing import StandardScaler
from pickle import dump
from pickle import load
from sklearn.preprocessing import StandardScaler
import mpl_toolkits.mplot3d

# Functions from @Mateen Ulhaq and @karlo
def set_axes_equal(ax: plt.Axes):
    """Set 3D plot axes to equal scale.

    Make axes of 3D plot have equal scale so that spheres appear as
    spheres and cubes as cubes.  Required since `ax.axis('equal')`
    and `ax.set_aspect('equal')` don't work on 3D.
    """
    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])
    origin = np.mean(limits, axis=1)
    radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
    _set_axes_radius(ax, origin, radius)

def _set_axes_radius(ax, origin, radius):
    x, y, z = origin
    ax.set_xlim3d([x - radius, x + radius])
    ax.set_ylim3d([y - radius, y + radius])
    ax.set_zlim3d([z - radius, z + radius])


In [ ]:
# tf.config.list_physical_devices()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.1)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))


In [ ]:
os.chdir("/data/raw_data")

ScanTime='10-05-2022 15.12.10.905'
CoordinateRaw = pd.read_csv('Coordinate_matched_'+ScanTime+'.csv')
SpectraRaw = pd.read_csv('Spectrum_matched_'+ScanTime+'.csv')

# print(type(CoordinateRaw))
# print(CoordinateRaw.shape)
# print(CoordinateRaw.iloc[0,42])
# print(CoordinateRaw.iloc[0,:])

print(type(SpectraRaw))
print(SpectraRaw.shape)
# print(SpectraRaw.iloc[0,42])
# print(SpectraRaw.iloc[0,:])
print(SpectraRaw.iloc[0:10,0])



In [ ]:
print(type(CoordinateRaw))
print(CoordinateRaw.shape)
print(CoordinateRaw.iloc[0,42])
print(CoordinateRaw.iloc[0,:])

In [ ]:
Spectra=SpectraRaw
Coordinate=CoordinateRaw
Spectra.iloc[:,0] =  pd.to_datetime(SpectraRaw.iloc[:,0], format='%Y-%m-%d %H:%M:%S.%f')
Spectra=Spectra.set_index('timestamps')
Coordinate.iloc[:,0] =  pd.to_datetime(CoordinateRaw.iloc[:,0], format='%Y-%m-%d %H:%M:%S.%f')
Coordinate=Coordinate.set_index('QualTime')
print(Spectra.shape)
print(Coordinate.shape)
Spectra = Spectra.iloc[:,0:191].astype('float64')
Spectra.info()
# print(Spectra.index)

In [ ]:
Time=Spectra.index
i = len(Time)-1
while i>=0:
    if(Time[i]<=Time[i-2]+datetime.timedelta(milliseconds=25)):#24 50
        print(i)
        i=i-3
    else:
        print("Removed", i)
        Time=np.delete(Time,i)
        Spectra=Spectra.drop(Spectra.index[i])
        Coordinate=Coordinate.drop(Coordinate.index[i])
        i=i-1

print(Time.shape)
print(Spectra.shape)
print(Coordinate.shape)

In [ ]:
k=1; # we remove this amount of rows for making the number of rows divisible
print(Spectra.shape)
print((len(Spectra)-k)/3)
# len(SpectraNarrow.columns)
# SpectraNarrow[0:len(SpectraNarrow)-2].shape

In [ ]:
n_rows=(len(Spectra)-k)
m_spectra=len(Spectra.columns)
Spectraflat=Spectra[0:n_rows].values.flatten()
print(Spectraflat.shape)
SpectraReshaped = np.reshape(Spectraflat, (int(n_rows/3),3,m_spectra))
print(SpectraReshaped.shape)
SpectraFinal=np.transpose(SpectraReshaped,(0,2,1))
SpectraFinal_flatten=SpectraFinal.flatten()
print(SpectraFinal.shape)
CoordinateFinal=Coordinate[0:n_rows].groupby(np.arange(n_rows)//3).mean()
CoordinateFinal=DataFrame.to_numpy(CoordinateFinal)
print (CoordinateFinal.shape)
print(n_rows/3)

In [ ]:
origin=[375.1,445,204.3]

CoordinateFinal_1=(CoordinateFinal.reshape((-1,20,3))-origin).reshape((-1,60))
x_train, x_VT, y_train, y_VT = train_test_split(SpectraFinal, CoordinateFinal_1, test_size=0.2, shuffle= True)
x_test, x_val, y_test, y_val = train_test_split(x_VT, y_VT, test_size=0.5, shuffle= True)

print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

print(x_test.shape)
print(y_test.shape)

In [ ]:
[n_tr,m_tr]=y_train.shape;
[n_vl,m_vl]=y_val.shape;

y_train_1 = np.zeros((n_tr,60)) # Pre-allocate matrix
y_val_1 = np.zeros((n_vl,60)) 

for i in range(0,n_tr):
    y_train_1[i,0:3]=y_train[i,0:3]
    for j in range(0,19):
            y_train_1[i,3*j+3]=y_train[i,3*j+3]-y_train[i,3*j]
            y_train_1[i,3*j+4]=y_train[i,3*j+4]-y_train[i,3*j+1]
            y_train_1[i,3*j+5]=y_train[i,3*j+5]-y_train[i,3*j+2]
            
for i in range(0,n_vl):
    y_val_1[i,0:3]=y_val[i,0:3]
    for j in range(0,19):
            y_val_1[i,3*j+3]=y_val[i,3*j+3]-y_val[i,3*j]
            y_val_1[i,3*j+4]=y_val[i,3*j+4]-y_val[i,3*j+1]
            y_val_1[i,3*j+5]=y_val[i,3*j+5]-y_val[i,3*j+2]


In [ ]:
std=StandardScaler()

x_train_std = std.fit_transform(x_train.reshape(-1, 1))
x_train = np.reshape(x_train_std, (-1,190,3))

x_test_std = std.transform(x_test.reshape(-1, 1))
x_test = np.reshape(x_test_std, (-1,190,3))

x_val_std = std.transform(x_val.reshape(-1, 1))
x_val = np.reshape(x_val_std, (-1,190,3))

x_train_flatten=x_train.flatten()

x_val_flatten=x_val.flatten()

x_test_flatten=x_test.flatten()

dump(std, open('scaler.pkl', 'wb')) # save the scalar

In [ ]:
i=1000
xyz = y_train_1[i].reshape((20,3))
spec = np.reshape(x_test, (-1,190,3))[i,:,:]

fig = plt.figure()
plt.plot(spec)
fig.show()

fig = plt.figure()
ax= fig.gca(projection='3d')
plt.ion()
# print(xyz)
ax.plot(xyz[:,0], xyz[:,1], xyz[:,2])
ax.set_box_aspect([1,1,1])
set_axes_equal(ax)
ax.set_xlabel('X [mm]')#, rotation=150, fontsize=12
ax.set_ylabel('Y [mm]')
ax.set_zlabel('Z [mm]')
fig.show()


In [ ]:
ScanTime='10-05-2022--15-12'
savetxt('x_test_Norm_Relative_1606Random_'+ScanTime+'.csv', x_test_flatten, delimiter=';')
savetxt('x_train_Norm_Relative_1606Random_'+ScanTime+'.csv', x_train_flatten, delimiter=';')
savetxt('x_val_Norm_Relative_1606Random_'+ScanTime+'.csv', x_val_flatten, delimiter=';')

savetxt('y_test_Norm_Relative_1606Random_'+ScanTime+'.csv', y_test, delimiter=';')
savetxt('y_train_Norm_Relative_1606Random_'+ScanTime+'.csv', y_train_1, delimiter=';')
savetxt('y_val_Norm_Relative_1606Random_'+ScanTime+'.csv', y_val_1, delimiter=';')

savetxt('x_test_NoPreProcessing_1606Random_'+ScanTime+'.csv', x_test_flatten, delimiter=';')
savetxt('x_train_NoPreProcessing_1606Random_'+ScanTime+'.csv', x_train_flatten, delimiter=';')
savetxt('x_val_NoPreProcessing_1606Random_'+ScanTime+'.csv', x_val_flatten, delimiter=';')

savetxt('y_test_NoPreProcessing_1606Random_'+ScanTime+'.csv', y_test, delimiter=';')
savetxt('y_train_NoPreProcessing_1606Random_'+ScanTime+'.csv', y_train, delimiter=';')
savetxt('y_val_NoPreProcessing_1606Random_'+ScanTime+'.csv', y_val, delimiter=';')
